In [1]:
%load_ext autoreload
%autoreload 2

# Camera-Depth calibration
Second version, only interesting the last part con Camera lidar calibration

In [12]:
# import mrob
import numpy as np
import open3d as o3d

In [3]:
pnp_results = np.load('calib_output/pnp_results.npy', allow_pickle=True).item()

In [6]:
item = pnp_results[list(pnp_results.keys())[1]] # Elements are ordered by an image name
T = item[0] # Transformation of points from uniform grid on chessboard space to camera space
t_scene_points = item[1] # Points from uniform grid on chessboard tranformed by applying T, i.e. T @ p

In [5]:
# pi = mrob.registration.estimate_plane(t_scene_points[0])
# print(pi)

In [11]:
x = o3d.geometry.PointCloud()
x.points = o3d.utility.Vector3dVector(t_scene_points)
#o3d.visualization.draw_geometries([x])
print(len(pnp_results))

106


# load pcd from velodyne

# Segment PC belonging to the plane

# Camera to Lidar calibration

In [22]:
import mrob
#initialization
N = 100 # len(pnp_results)
graph = mrob.FGraph()
W = np.array([1])
n1 = graph.add_node_pose_3d(mrob.geometry.SE3())

In [23]:
for n in range(N):
    #load PnP
    item = pnp_results[list(pnp_results.keys())[n]]
    Y = item[1] #point cloud
    normal = mrob.registration.estimate_normal(X)
    #Y_center = mrob.registration.estimate_centroid(X)
    #print(normal)
    
    #load pcd
    item = pnp_results[list(pnp_results.keys())[n]]
    X = item[1] #centroid of 
    N_points = Y.shape[0]
    
    #create factors
    for i in range(N_points):
        graph.add_factor_1pose_point2plane(z_point_x = X[i],
                                       z_point_y = Y[i], #Y_center if it is not working well
                                       z_normal_y = normal,
                                       nodePoseId = n1,
                                       obsInf = W)
# solve the problem
graph.solve(mrob.LM)
result = graph.get_estimated_state()
print(result[0])




[[ 9.98223192e-01 -2.59456367e-02  5.36403106e-02 -5.49411637e-02]
 [ 2.59418229e-02  9.99663160e-01  7.67481528e-04  1.55760996e-02]
 [-5.36421552e-02  6.25409578e-04  9.98560027e-01  1.58697725e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [24]:
# test visually
pcx = o3d.geometry.PointCloud()
pcx.points= o3d.utility.Vector3dVector(X)
pcy = o3d.geometry.PointCloud()
pcy.points= o3d.utility.Vector3dVector(Y)
pcx.transform(result[0])
o3d.visualization.draw_geometries([pcx,pcy])